Se importan las librerias y paquetes requidos

In [457]:
import pandas as pd
import numpy as np 
import json
import ast
from sklearn.preprocessing import LabelEncoder

In [458]:
df = pd.read_csv('./origen.csv')
test = pd.read_csv('./testear.csv')

# Limpieza de datos
## Variables de tipo numericas
- average rating
- numVotes
- isAdult
- startYear
- endYear
- runtimeMinutes
- seasonNumber
- episodeNumber
- ordering
- language
- isOriginalTitle
- budget
- popularity
- revenue
- runtime

## Variables de tipo str
- titleType
- genres_x
- directors
- writers
- adult
- genres_y
- original_language
- production_company
- production_country
- status
- tagline
- video

## Variables booleanas

Todas las columnas son rotuladas de manera numerica
Las columnas indice, directors, writer, tagline fueron eliminadas

In [459]:
df = df.drop('Unnamed: 0', axis=1)

In [460]:
print(df['genres_x'].str.count(',').max())
# Se pueden tener como maximo 3 generos x

df[['generox1', 'generox2', 'generox3']] = df['genres_x'].str.split(',', expand=True)
df = df.drop('genres_x', axis = 1)

2.0


In [461]:
generox = LabelEncoder()
df["generox1_encoded"] = generox.fit_transform(df["generox1"])
df["generox2_encoded"] = generox.fit_transform(df["generox2"])
df["generox3_encoded"] = generox.fit_transform(df["generox3"])
df['generox1'] = df['generox1_encoded']
df['generox2'] = df['generox2_encoded']
df['generox3'] = df['generox3_encoded']
df = df.drop('generox1_encoded', axis = 1)
df = df.drop('generox2_encoded', axis = 1)
df = df.drop('generox3_encoded', axis = 1)

In [462]:
leAdult = LabelEncoder()
df["isAdult_encoded"] = leAdult.fit_transform(df["isAdult"])
df['isAdult_encoded'].fillna("NA", inplace=True)
df['isAdult'] = df['isAdult_encoded']
df = df.drop('isAdult_encoded', axis = 1)

df = df.drop('adult', axis = 1)

In [463]:
leTitleType = LabelEncoder()
df["titleType_encoded"] = leTitleType.fit_transform(df["titleType"])
df['titleType'] = df['titleType_encoded']
df = df.drop('titleType_encoded', axis = 1)

In [464]:
leLanguage = LabelEncoder()
df["language_encoded"] = leLanguage.fit_transform(df["original_language"])
df['original_language'] = df['language_encoded']
df = df.drop('language_encoded', axis = 1)

df = df.drop('language', axis = 1)

In [465]:
leStatus = LabelEncoder()
df["status_encoded"] = leLanguage.fit_transform(df["status"])
df['status'] = df['status_encoded']
df = df.drop('status_encoded', axis = 1)

In [466]:
leVideo = LabelEncoder()
df["video_encoded"] = leAdult.fit_transform(df["video"])
df['video'] = df['video_encoded']
df = df.drop('video_encoded', axis = 1)

In [467]:
leAttributes = LabelEncoder()
df["attributes_encoded"] = leAdult.fit_transform(df["attributes"])
df['attributes'] = df['attributes_encoded']
df = df.drop('attributes_encoded', axis = 1)

In [468]:
print(df['writers'].str.count(',').max())
# Se pueden tener hasta 1288 escritores

print(df['directors'].str.count(',').max())
# Se pueden tener hasta 475 directores

df = df.drop('writers', axis = 1)
df = df.drop('directors', axis = 1)

1287
475


In [469]:
df = df.drop('tagline', axis = 1)

In [470]:
def extract_name(row):
    if isinstance(row, list) and len(row) > 0:
        company_dict = row[0]
        if 'name' in company_dict:
            return company_dict['name']
    return ""

In [471]:
object_columns = df.select_dtypes(include=['object']).columns
print(object_columns)

Index(['genres_y', 'production_companies', 'production_countries'], dtype='object')


In [472]:
def extract_and_join_names(row):
    if pd.notna(row):  
        cleaned_json = row.replace("'", '"').strip() 
        try:
            country_list = json.loads(cleaned_json)
            names = [country['name'] for country in country_list]
            return ', '.join(names)
        except json.JSONDecodeError as e:
            return 'Invalid JSON'
    else:
        return ''

In [473]:
df['countries'] = df['production_countries'].apply(extract_and_join_names)

In [474]:
df['companies'] = df['production_companies'].apply(extract_and_join_names)

In [475]:
df['genre_y'] = df['genres_y'].apply(extract_and_join_names)

In [476]:
def extract_column_list(column, key):
    keys = []

    for i in range(df[column].str.count(', ').max() + 1):
        keys.append(key.format(n=i))

    df[keys] = df[column].str.split(',', expand=True)

In [477]:
extract_column_list('genre_y', 'genre_y_{n:d}')
df.drop('genre_y', axis=1, inplace=True)
df.drop('genres_y', axis=1, inplace=True)

In [478]:
extract_column_list('companies', 'companies_{n:d}')
df.drop('companies', axis=1, inplace=True)
df.drop('production_companies', axis=1, inplace=True)

In [479]:
extract_column_list('countries', 'countries_{n:d}')
df.drop('countries', axis=1, inplace=True)
df.drop('production_countries', axis=1, inplace=True)

KeyError: "['genre_y'] not found in axis"

In [ ]:
key = 'genre_y_{n:d}'
key_encoded = 'genre_y_{n:d}_encoded'

for i in range(df[key].str.count(', ').max() + 1):
    le = LabelEncoder()
    df[key_encoded.format(n=i)] = le.fit_transform(df[key.format(n=i)])
    df[key.format(n=i)] = df[key_encoded.format(n=i)]
    df = df.drop(key_encoded.format(n=i), axis = 1) 

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'seasonNumber', 'episodeNumber',
       'ordering', 'attributes', 'isOriginalTitle', 'budget', 'genres_y',
       'original_language', 'popularity', 'production_companies',
       'production_countries', 'revenue', 'runtime', 'status', 'video',
       'generox1', 'generox2', 'generox3', 'countries', 'companies',
       'genre_y_0', 'genre_y_1', 'genre_y_2', 'genre_y_3', 'genre_y_4',
       'genre_y_5', 'genre_y_6', 'genre_y_7', 'companies_0', 'companies_1',
       'companies_2', 'companies_3', 'companies_4', 'companies_5',
       'companies_6', 'companies_7', 'companies_8', 'companies_9',
       'companies_10', 'companies_11', 'companies_12', 'companies_13',
       'companies_14', 'companies_15', 'companies_16', 'companies_17',
       'companies_18', 'companies_19', 'companies_20', 'companies_21',
       'companies_22', 'companies_23', 'companies_24', 'companies_25',
       'cou